In [86]:
'''
this script takes as input the LSTM or RNN weights found by train.py
change the path in line 176 of this script to point to the h5 file
with LSTM or RNN weights generated by train.py
'''

from __future__ import division
from keras.models import load_model
import csv
import copy
import numpy as np
import distance
from jellyfish._jellyfish import damerau_levenshtein_distance
import csv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

In [87]:
eventlog = 'event_data_alexander_rounded.csv'
csvfile = open('data/%s' % eventlog, 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|')

#helper variables
lastcase = ''
firstLine = True
lineseqs = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
caseidseqs = []
lines = []
caseids = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None
next(datareader, None)  # skip the headers
for row in datareader: #the columns are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            caseidseqs.append(caseids)
            lineseqs.append(lines)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
        caseids = []
        lines = []
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds     #multiply with 60*60*24 = 86400 to go from days to seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds    #the .seconds method gives the time in seconds
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    caseids.append(row[0])
    lines.append(str(row[1])) #add the activity label to the line list
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    times4.append(timediff4)
    lasteventtime = t
    firstLine = False


# add last case
elems_per_fold = int(round(numlines/3))
caseidseqs.append(caseids)
lineseqs.append(lines)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
numlines+=1

divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))
divisor3 = np.mean(
list(map(lambda x: np.mean(list(map(lambda y: x[len(x)-1]-y, x))), timeseqs2)))
print('divisor3: {}'.format(divisor3))

divisor: 8329.618428334255
divisor2: 257467.7155506364
divisor3: 288305.9640935422


In [88]:
lines

['1',
 '10',
 '10',
 '1',
 '9',
 '13',
 '14',
 '13',
 '13',
 '13',
 '13',
 '15',
 '13',
 '15',
 '9',
 '7',
 '7',
 '4',
 '3',
 '13',
 '13',
 '15',
 '13',
 '14',
 '14',
 '13',
 '14',
 '13',
 '13',
 '13',
 '15',
 '13',
 '14',
 '13',
 '13',
 '8',
 '13',
 '13',
 '5',
 '3',
 '6',
 '8',
 '15',
 '6',
 '13',
 '13',
 '13',
 '15',
 '12',
 '13',
 '15',
 '13',
 '14',
 '13',
 '13',
 '13',
 '13',
 '13',
 '15',
 '13',
 '15',
 '13',
 '13',
 '14',
 '13',
 '13',
 '13',
 '13',
 '15',
 '13',
 '15',
 '12']

In [89]:
elems_per_fold = int(round(numlines/3))
fold1 = lineseqs[:elems_per_fold]
fold1_c = caseids[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

fold2 = lineseqs[elems_per_fold:2*elems_per_fold]
fold2_c = caseids[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

lines = fold1 + fold2
caseids = fold1_c + fold2_c
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

step = 1
sentences = []
softness = 0
next_chars = []
maxlen = 119

activities = set([item for sublist in lines for item in sublist])
target_activities = copy.copy(activities)

print('total activities: {}, target activities: {}'.format(len(activities), len(target_activities)))
act_indices = dict((c, i) for i, c in enumerate(activities))
indices_act = dict((i, c) for i, c in enumerate(activities))
target_activities_indices = dict((c, i) for i, c in enumerate(target_activities))
target_indices_activities = dict((i, c) for i, c in enumerate(target_activities))
print(indices_act)

lastcase = ''
firstLine = True
lines = []
caseids = []
timeseqs = []  # relative time since previous event
timeseqs2 = [] # relative time since case start
timeseqs3 = [] # absolute time of previous event
lineseqs = []
times = []
times2 = []
times3 = []
numlines = 0
casestarttime = None
lasteventtime = None

eventlog = 'event_data_alexander_rounded.csv'
csvfile = open('data/%s' % eventlog, 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(datareader, None)  # skip the headers
for row in datareader:
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        caseids.append(row[0])
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lineseqs.append(lines)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
        line = ''
        times = []
        numlines+=1
    lines.append(int(row[1]))
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    #timediff = log(timediff+1)
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    lasteventtime = t
    firstLine = False

# add last case
lineseqs.append(lines)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
numlines+=1

fold3 = lineseqs[2*elems_per_fold:]
fold3_c = caseids[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
#fold3_t4 = timeseqs4[2*elems_per_fold:]

lines = fold3
caseids = fold3_c
lines_t = fold3_t
lines_t2 = fold3_t2
lines_t3 = fold3_t3
#lines_t4 = fold1_t4 + fold2_t4

# set parameters
predict_size = 1

# load model, set this to the model generated by train.py
model = load_model('output_files/models/model_97-1.38.h5')

# define helper functions
def encode(sentence, times, times3, maxlen=maxlen):
    num_features = len(activities)+5
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in activities:
            if c==char:
                X[0, t+leftpad, act_indices[c]] = 1
        X[0, t+leftpad, len(activities)] = t+1
        X[0, t+leftpad, len(activities)+1] = times[t]/divisor
        X[0, t+leftpad, len(activities)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(activities)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(activities)+4] = times3[t].weekday()/7
    return X

def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_activities[i]
        i += 1
    return symbol

one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []

total activities: 15, target activities: 15
{0: '15', 1: '4', 2: '2', 3: '10', 4: '9', 5: '6', 6: '11', 7: '14', 8: '8', 9: '13', 10: '7', 11: '3', 12: '12', 13: '5', 14: '1'}


In [91]:
# make predictions
with open('output_files/results/next_activity_and_time_%s' % eventlog, 'w') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    datawriter.writerow(["CaseID", "Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE"])
    for prefix_size in range(2,10):
        print(prefix_size)
        for line, caseid, times, times3 in zip(lines, caseids, lines_t, lines_t3):
            times.append(0)
            cropped_line = line[:prefix_size]
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if '!' in cropped_line:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = line[prefix_size:prefix_size+predict_size]
            ground_truth_t = times[prefix_size:prefix_size+predict_size]
            predicted = ''
            predicted_t = []
            for i in range(predict_size):
                if len(ground_truth)<=i:
                    continue
                enc = encode(cropped_line, cropped_times, cropped_times3)
                y = model.predict(enc, verbose=0)
                y_char = y[0][0]
                y_t = y[1][0][0]
                prediction = getSymbol(y_char)              
                cropped_line += prediction
                if y_t<0:
                    y_t=0
                cropped_times.append(y_t)
                y_t = y_t * divisor
                cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                predicted_t.append(y_t)
                if i==0:
                    if len(ground_truth_t)>0:
                        one_ahead_pred.append(y_t)
                        one_ahead_gt.append(ground_truth_t[0])
                if i==1:
                    if len(ground_truth_t)>1:
                        two_ahead_pred.append(y_t)
                        two_ahead_gt.append(ground_truth_t[1])
                if i==2:
                    if len(ground_truth_t)>2:
                        three_ahead_pred.append(y_t)
                        three_ahead_gt.append(ground_truth_t[2])
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    print('! predicted, end case')
                    break
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(caseid)
                output.append(prefix_size)
                output.append(str(ground_truth).encode("utf-8"))
                output.append(str(predicted).encode("utf-8"))
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(str(predicted), str(ground_truth)) / max(len(predicted),len(ground_truth)))
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                output.append(1 - distance.jaccard(predicted, ground_truth))
                output.append('; '.join(str(x) for x in ground_truth_t))
                output.append('; '.join(str(x) for x in predicted_t))
                if len(predicted_t)>len(ground_truth_t): # if predicted more events than length of case, only use needed number of events for time evaluation
                    predicted_t = predicted_t[:len(ground_truth_t)]
                if len(ground_truth_t)>len(predicted_t): # if predicted less events than length of case, put 0 as placeholder prediction
                    predicted_t.extend(range(len(ground_truth_t)-len(predicted_t)))
                if len(ground_truth_t)>0 and len(predicted_t)>0:
                    output.append('')
                    output.append(metrics.mean_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                    output.append(metrics.median_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                else:
                    output.append('')
                    output.append('')
                    output.append('')
                datawriter.writerow(output)

2
3
4
5
6
7
8
9


In [92]:
import pandas as pd
results = pd.read_csv(r"G:\My Drive\CurrentWork\NEP\IoTPPM\output_files\results\next_activity_and_time_event_data_alexander_rounded.csv")
pd.set_option('display.max_rows', None)
results = results.sort_values(['CaseID','Prefix length'])
results['CaseID']= results.index
results = results.reindex()
print(results.head())

     CaseID Prefix length Groud truth  Predicted  Levenshtein  Damerau  \
117     117       b'[10]'       b'11'        0.0          0.0      0.0   
118     118       b'[10]'        b'7'        0.0          0.0      0.0   
119     119       b'[10]'       b'11'        0.0          0.0      0.0   
120     120       b'[10]'       b'11'        0.0          0.0      0.0   
121     121       b'[10]'        b'7'        0.0          0.0      0.0   

     Jaccard  Ground truth times  Predicted times         RMSE          MAE  
117     1351         1569.610417              NaN   218.610417   218.610417  
118      449         2018.740470              NaN  1569.740470  1569.740470  
119     1554         1624.760971              NaN    70.760971    70.760971  
120     5958         1639.474894              NaN  4318.525106  4318.525106  
121     6559         1728.669104              NaN  4830.330896  4830.330896  
